In [1]:
import os
from pathlib import Path
import pandas as pd
from entsoe import EntsoePandasClient

from config import (
    TZ, COUNTRY_CODE, DAYS_BACK, BACKFILL_DAYS, INITIAL_HISTORY_DAYS,
    SITE_TITLE, SITE_TAGLINE, OUTPUT_DIR, OUTPUT_FILE, PARQUET_FILE, 
    DATA_DIR, ZONE_CODES, TARGET_ZONES
)

from data_fetch import fetch_load_df, get_time_range, to_display_df, to_storage_df, update_history_parquet_multi
from plotting import make_actual_load_plot, make_all_zones_plot
from page_builder import build_page

# Load ENTSOE_API_KEY from a local .env file
try:
    from dotenv import load_dotenv
    load_dotenv()
except Exception:
    pass


api_key = os.getenv("ENTSOE_API_KEY")
if not api_key:
    raise SystemExit("Missing ENTSOE_API_KEY")

# Decide how much to fetch: first run = long history, otherwise small backfill
if not PARQUET_FILE.exists():
    start, end = get_time_range(INITIAL_HISTORY_DAYS)
else:
    start, end = get_time_range(BACKFILL_DAYS)

client = EntsoePandasClient(api_key=api_key)

In [2]:
storage_chunks = []

for zl in TARGET_ZONES:
        
    code = ZONE_CODES[zl]
    display_df = fetch_load_df(client, code, start, end)  # -> ["Date","Load (MW)"]
    storage_chunks.append(to_storage_df(display_df, zone_label=zl))  # -> ["date","zone","load_mw"]